In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.options.display.max_columns=30
pd.options.display.max_rows=30

In [3]:
df_listings=pd.read_csv('listings.csv')
df_listings=df_listings.rename(columns={'id':'listing_id'})
df_listings.head()

,listing_id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,...,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2818,https://www.airbnb.com/rooms/2818,20220308023209,2022-03-08,Quiet Garden View Room & Super Fast WiFi,Quiet Garden View Room & Super Fast WiFi<br />...,"Indische Buurt (""Indies Neighborhood"") is a ne...",https://a0.muscache.com/pictures/10272854/8dcc...,3159,https://www.airbnb.com/users/show/3159,Daniel,2008-09-24,"Amsterdam, Noord-Holland, The Netherlands","Upon arriving in Amsterdam, one can imagine as...",within an hour,...,2021-11-21,4.89,4.93,5.00,4.97,4.98,4.68,4.82,0363 5F3A 5684 6750 D14D,t,1,0,1,0,1.81
1,20168,https://www.airbnb.com/rooms/20168,20220308023209,2022-03-31,Studio with private bathroom in the centre 1,17th century Dutch townhouse in the heart of t...,Located just in between famous central canals....,https://a0.muscache.com/pictures/69979628/fd6a...,59484,https://www.airbnb.com/users/show/59484,Alexander,2009-12-02,"Amsterdam, Noord-Holland, The Netherlands",+ (Phone number hidden by Airbnb),within an hour,...,2020-04-09,4.44,4.69,4.79,4.63,4.62,4.87,4.49,0363 CBB3 2C10 0C2A 1E29,t,2,0,2,0,2.30
2,27886,https://www.airbnb.com/rooms/27886,20220308023209,2022-03-08,"Romantic, stylish B&B houseboat in canal district",Stylish and romantic houseboat on fantastic hi...,"Central, quiet, safe, clean and beautiful.",https://a0.muscache.com/pictures/02c2da9d-660e...,97647,https://www.airbnb.com/users/show/97647,Flip,2010-03-23,"Amsterdam, Noord-Holland, The Netherlands","Marjan works in ""eye"" the dutch filmmuseum, an...",within an hour,...,2022-02-20,4.95,4.93,4.96,4.95,4.92,4.89,4.79,0363 974D 4986 7411 88D8,t,1,0,1,0,1.84
3,28871,https://www.airbnb.com/rooms/28871,20220308023209,2022-03-08,Comfortable double room,<b>The space</b><br />In a monumental house ri...,"Flower market , Leidseplein , Rembrantsplein",https://a0.muscache.com/pictures/160889/362340...,124245,https://www.airbnb.com/users/show/124245,Edwin,2010-05-13,"Amsterdam, Noord-Holland, The Netherlands",Hi,within an hour,...,2022-03-05,4.88,4.94,4.89,4.96,4.94,4.97,4.82,0363 607B EA74 0BD8 2F6F,f,2,0,2,0,2.70
4,29051,https://www.airbnb.com/rooms/29051,20220308023209,2022-03-08,Comfortable single room,This room can also accomodate 2 people. For a...,the street is quite lively especially on weeke...,https://a0.muscache.com/pictures/162009/bd6be2...,124245,https://www.airbnb.com/users/show/124245,Edwin,2010-05-13,"Amsterdam, Noord-Holland, The Netherlands",Hi,within an hour,...,2022-03-04,4.78,4.87,4.84,4.92,4.91,4.87,4.75,0363 607B EA74 0BD8 2F6F,f,2,0,2,0,3.98


In [4]:
df_listings.columns

Index(['listing_id', 'listing_url', 'scrape_id', 'last_scraped', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'cale

In [5]:
columns_needed=['listing_id','property_type','room_type','accommodates',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price']
df=df_listings[columns_needed]
df.head(20)

,listing_id,property_type,room_type,accommodates,bathrooms_text,bedrooms,beds,amenities,price
0,2818,Private room in rental unit,Private room,2,1.5 shared baths,1.0,2.0,"[""Shampoo"", ""Laundromat nearby"", ""Paid parking...",$49.00
1,20168,Private room in townhouse,Private room,2,1 private bath,1.0,1.0,"[""Essentials"", ""Hair dryer"", ""Hangers"", ""Dedic...",$106.00
2,27886,Private room in houseboat,Private room,2,1.5 baths,1.0,1.0,"[""Shampoo"", ""Cable TV"", ""Laundromat nearby"", ""...",$134.00
3,28871,Private room in rental unit,Private room,2,1 shared bath,1.0,1.0,"[""Refrigerator"", ""Shampoo"", ""Private entrance""...",$75.00
4,29051,Private room in rental unit,Private room,1,1 shared bath,1.0,1.0,"[""Refrigerator"", ""Shampoo"", ""Private entrance""...",$55.00
5,44391,Entire rental unit,Entire home/apt,4,1.5 baths,2.0,2.0,"[""Refrigerator"", ""Hangers"", ""Cooking basics"", ...",$240.00
6,47061,Entire rental unit,Entire home/apt,3,1.5 baths,2.0,2.0,"[""Shampoo"", ""Cable TV"", ""Kitchen"", ""Hot water ...",$140.00
7,49552,Entire guest suite,Entire home/apt,3,1 bath,2.0,2.0,"[""Children\u2019s dinnerware"", ""Babysitter rec...",$187.00
8,49790,Boat,Entire home/apt,4,1 bath,2.0,2.0,"[""Shampoo"", ""Essentials"", ""Dryer"", ""Cable TV"",...",$250.00
9,50523,Private room in bed and breakfast,Private room,2,1 private bath,1.0,1.0,"[""Private entrance"", ""Breakfast"", ""Essentials""...",$114.00


In [6]:
data_missing = df.isna()
data_missing = data_missing.sum()
data_missing = data_missing / len(df)
print("Missing field rate (sorted):")
data_missing[data_missing > 0.0].sort_values(ascending=False)

Missing field rate (sorted):


bedrooms          0.057048
beds              0.017620
bathrooms_text    0.003140
dtype: float64

In [7]:
df['bedrooms']=df['bedrooms'].fillna(df['bedrooms'].mode()[0])
df['beds']=df['beds'].fillna(df['beds'].mode()[0])
df['bathrooms_text']=df['bathrooms_text'].fillna(df['bathrooms_text'].mode()[0])

In [8]:
df['amenities'][0]

'["Shampoo", "Laundromat nearby", "Paid parking off premises", "Clothing storage: wardrobe", "Hot water kettle", "Shower gel", "Carbon monoxide alarm", "Long term stays allowed", "Coffee maker", "Paid washer \\u2013 In unit", "Hot water", "Single level home", "Drying rack for clothing", "Security cameras on property", "Cleaning products", "Fire extinguisher", "Hair dryer", "Smoke alarm", "Pour-over coffee", "Dedicated workspace: table", "Room-darkening shades", "Mini fridge", "Ethernet connection", "First aid kit", "Portable fans", "Body soap", "Extra pillows and blankets", "Lock on bedroom door", "Bed linens", "Heating", "Mosquito net", "Iron", "Paid parking on premises", "Refrigerator", "Private entrance", "Fast wifi \\u2013 52 Mbps", "Essentials", "Outdoor furniture", "Host greets you", "Shared fenced garden or backyard", "Hangers"]'

In [9]:
length=[]
for i in df['amenities']:
    j='"'
    count=i.count(j)
    length.append(count//2)
df['total_amenities']=pd.DataFrame(length)


In [10]:
df=df.drop(columns=['amenities'])
df.head(10)

,listing_id,property_type,room_type,accommodates,bathrooms_text,bedrooms,beds,price,total_amenities
0,2818,Private room in rental unit,Private room,2,1.5 shared baths,1.0,2.0,$49.00,41
1,20168,Private room in townhouse,Private room,2,1 private bath,1.0,1.0,$106.00,17
2,27886,Private room in houseboat,Private room,2,1.5 baths,1.0,1.0,$134.00,44
3,28871,Private room in rental unit,Private room,2,1 shared bath,1.0,1.0,$75.00,20
4,29051,Private room in rental unit,Private room,1,1 shared bath,1.0,1.0,$55.00,19
5,44391,Entire rental unit,Entire home/apt,4,1.5 baths,2.0,2.0,$240.00,34
6,47061,Entire rental unit,Entire home/apt,3,1.5 baths,2.0,2.0,$140.00,42
7,49552,Entire guest suite,Entire home/apt,3,1 bath,2.0,2.0,$187.00,46
8,49790,Boat,Entire home/apt,4,1 bath,2.0,2.0,$250.00,16
9,50523,Private room in bed and breakfast,Private room,2,1 private bath,1.0,1.0,$114.00,19


In [11]:
#df['price'] = df['price'].astype(str)
df['price']=df['price'].str.replace("$","")
df['price']=df['price'].str.replace(",","")
df['price']=df['price'].astype(float)
df['bedrooms']=df['bedrooms'].astype(float)
df['bedrooms']=df['bedrooms'].astype(int)
df['beds']=df['beds'].astype(float)
df['beds']=df['beds'].astype(int)

In [12]:
df.head()

,listing_id,property_type,room_type,accommodates,bathrooms_text,bedrooms,beds,price,total_amenities
0,2818,Private room in rental unit,Private room,2,1.5 shared baths,1,2,49.0,41
1,20168,Private room in townhouse,Private room,2,1 private bath,1,1,106.0,17
2,27886,Private room in houseboat,Private room,2,1.5 baths,1,1,134.0,44
3,28871,Private room in rental unit,Private room,2,1 shared bath,1,1,75.0,20
4,29051,Private room in rental unit,Private room,1,1 shared bath,1,1,55.0,19


In [13]:
pd.set_option('display.max_colwidth', 0)
df[['property_type','room_type','accommodates','bathrooms_text','bedrooms','beds']].agg(['unique']).transpose()

,unique
property_type,"[Private room in rental unit, Private room in townhouse, Private room in houseboat, Entire rental unit, Entire guest suite, Boat, Private room in bed and breakfast, Entire loft, Private room in home, Entire guesthouse, Private room in boat, Entire townhouse, Room in boutique hotel, Entire condo, Private room in loft, Private room in guest suite, Entire residential home, Entire condominium (condo), Private room in residential home, Entire home, Private room in farm stay, Houseboat, Room in bed and breakfast, Entire serviced apartment, Entire chalet, Private room in guesthouse, Shared room in rental unit, Barn, Private room in condominium (condo), Private room in condo, Shared room in residential home, Private room in island, Entire cottage, Farm stay, Entire villa, Private room in villa, Private room in tiny house, Private room in bungalow, Entire cabin, Room in hotel, Entire place, Private room in serviced apartment, Room in aparthotel, Tiny house, Shared room in bed and breakfast, Casa particular, Private room in cabin, Entire bungalow, Private room, Private room in nature lodge, Private room in tiny home, Room in serviced apartment, Tower, Room in casa particular, Room in hostel, Shared room in hostel, Private room in hostel, Bus, Shared room in boat, Camper/RV, Cave, Private room in barn, Entire home/apt, Private room in casa particular, Entire vacation home, Shared room in aparthotel]"
room_type,"[Private room, Entire home/apt, Hotel room, Shared room]"
accommodates,"[2, 1, 4, 3, 12, 16, 5, 6, 7, 14, 8, 9, 13, 10, 15, 0]"
bathrooms_text,"[1.5 shared baths, 1 private bath, 1.5 baths, 1 shared bath, 1 bath, 2.5 baths, 2 baths, 0 baths, Private half-bath, 3 baths, 3.5 baths, 0 shared baths, 5 baths, Half-bath, Shared half-bath, 3.5 shared baths, 3 shared baths, 2 shared baths, 4 baths, 5.5 baths, 4.5 baths, 4 shared baths]"
bedrooms,"[1, 2, 4, 5, 10, 3, 7, 6, 8, 15]"
beds,"[2, 1, 4, 3, 12, 6, 5, 24, 8, 16, 7, 14, 20, 18, 34, 17, 13, 10, 33, 9, 11]"


In [14]:
df['property_type'].value_counts()

Entire rental unit                   2346
Private room in rental unit          555 
Private room in bed and breakfast    291 
Entire residential home              207 
Entire condominium (condo)           192 
                                    ...  
Private room in bungalow             1   
Private room in tiny home            1   
Private room in nature lodge         1   
Entire bungalow                      1   
Shared room in aparthotel            1   
Name: property_type, Length: 66, dtype: int64

In [15]:
dict1 = {'Apartment/house':['Entire rental unit','Entire residential home','Entire condominium (condo)','Entire townhouse','Entire home','Entire loft','Entire condo',
                        'Private room in rental unit','Private room in residential home','Private room in townhouse','Entire home/apt','Shared room in residential home',
                           'Private room in condominium (condo)','Private room in loft','Private room in home','Private room in tiny home',
                           'Private room in tiny house','Entire place','Tiny house','Entire vacation home','Private room in villa','Entire cottage','Shared room in rental unit',
                           'Private room in guesthouse','Private room in condo','Entire villa','Entire guesthouse'],
         'Apartment':['Private room','Entire serviced apartment','Room in serviced apartment','Private room in serviced apartment',''],
         'Hotel':['Room in boutique hotel','Room in hotel','Private room in guest suite','Private room in bed and breakfast',
                'Room in aparthotel','Entire guest suite','Room in bed and breakfast','Shared room in aparthotel','Shared room in bed and breakfast'],
         'Hostel': ['Private room in hostel','Room in hostel','Shared room in hostel'],       
         'Boat' : ['Private room in boat','Private room in houseboat','Shared room in boat','Boat','Houseboat'],
         'Other':['Entire bungalow','Private room in bungalow','Private room in nature lodges','Private room in barn','Private room in nature lodge','Cave','Private room in island','Bus','Casa particular','Room in casa particular',
                'Entire chalet','Private room in cabin','Camper/RV','Tower','Barn','Farm stay','Entire cabin','Private room in farm stay','Private room in casa particular']
        }
dict2 = {i : k for k, v in dict1.items() for i in v}
df['property_type']=df['property_type'].replace(dict2)



In [16]:
df.head()

,listing_id,property_type,room_type,accommodates,bathrooms_text,bedrooms,beds,price,total_amenities
0,2818,Apartment/house,Private room,2,1.5 shared baths,1,2,49.0,41
1,20168,Apartment/house,Private room,2,1 private bath,1,1,106.0,17
2,27886,Boat,Private room,2,1.5 baths,1,1,134.0,44
3,28871,Apartment/house,Private room,2,1 shared bath,1,1,75.0,20
4,29051,Apartment/house,Private room,1,1 shared bath,1,1,55.0,19


In [17]:
df[['property_type','room_type','accommodates','bathrooms_text','bedrooms','beds']].agg(['unique']).transpose()

,unique
property_type,"[Apartment/house, Boat, Hotel, Other, Apartment, Hostel]"
room_type,"[Private room, Entire home/apt, Hotel room, Shared room]"
accommodates,"[2, 1, 4, 3, 12, 16, 5, 6, 7, 14, 8, 9, 13, 10, 15, 0]"
bathrooms_text,"[1.5 shared baths, 1 private bath, 1.5 baths, 1 shared bath, 1 bath, 2.5 baths, 2 baths, 0 baths, Private half-bath, 3 baths, 3.5 baths, 0 shared baths, 5 baths, Half-bath, Shared half-bath, 3.5 shared baths, 3 shared baths, 2 shared baths, 4 baths, 5.5 baths, 4.5 baths, 4 shared baths]"
bedrooms,"[1, 2, 4, 5, 10, 3, 7, 6, 8, 15]"
beds,"[2, 1, 4, 3, 12, 6, 5, 24, 8, 16, 7, 14, 20, 18, 34, 17, 13, 10, 33, 9, 11]"


In [18]:
df=pd.get_dummies(df, prefix=['property_type', 'room_type','bathrooms_text'])

In [19]:
df.head()

,listing_id,accommodates,bedrooms,beds,price,total_amenities,property_type_Apartment,property_type_Apartment/house,property_type_Boat,property_type_Hostel,property_type_Hotel,property_type_Other,room_type_Entire home/apt,room_type_Hotel room,room_type_Private room,...,bathrooms_text_2 baths,bathrooms_text_2 shared baths,bathrooms_text_2.5 baths,bathrooms_text_3 baths,bathrooms_text_3 shared baths,bathrooms_text_3.5 baths,bathrooms_text_3.5 shared baths,bathrooms_text_4 baths,bathrooms_text_4 shared baths,bathrooms_text_4.5 baths,bathrooms_text_5 baths,bathrooms_text_5.5 baths,bathrooms_text_Half-bath,bathrooms_text_Private half-bath,bathrooms_text_Shared half-bath
0,2818,2,1,2,49.0,41,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,20168,2,1,1,106.0,17,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,27886,2,1,1,134.0,44,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,28871,2,1,1,75.0,20,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,29051,1,1,1,55.0,19,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
X=df.drop(columns=['listing_id','price'],axis=1)
y=df['price']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size =0.3, random_state=13)

In [22]:

 
 # create regressor object
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
 
# fit the regressor with x and y data
regressor.fit(X_train, y_train)

prediction=regressor.predict(X_test)

mse = mean_squared_error(y_test, prediction)
rmse = mse**.5

print(mse)
print(rmse)

75758.33765698
275.2423253371109
